In [1]:
from FourierCKKS import *

In [3]:
poly_degree = 1024
message1_length = 256
message2_length = 200
message_out_length = message1_length + message2_length - 1
fcks = FourierCKKS(poly_degree=poly_degree)
orig1 = np.random.random(message1_length) + 1j * np.random.random(message1_length)
orig2 = np.random.random(message2_length) + 1j * np.random.random(message2_length)
ct1 = fcks.forward(orig1, message_out_length)
ct2 = fcks.forward(orig2, message_out_length)
rec1 = fcks.backward(ct1, message_out_length)
rec2 = fcks.backward(ct2, message_out_length)
success1 = np.allclose(rec1[:message1_length], orig1, atol=1e-1, rtol=1e-1)
success2 = np.allclose(rec2[:message2_length], orig2, atol=1e-1, rtol=1e-1)
print(f"Recovery1 successful: {success1}, Recovery2 successful: {success2}")

ct_sum = fcks.cipher_add(ct1, ct2)
rec_sum = fcks.backward(ct_sum, message_out_length)
true_sum = orig1.copy()
true_sum[:message2_length] += orig2
add_success = np.allclose(rec_sum[:message1_length], true_sum, atol=1e-1, rtol=1e-1)
print(f"Addition recovery successful: {add_success}")

ct_conv = fcks.cipher_conv(ct1, ct2)
rec_conv = fcks.backward(ct_conv, message_out_length)
from fft.fft import linear_convolution_direct
true_conv = linear_convolution_direct(orig1, orig2)
conv_success = np.allclose(rec_conv, true_conv, atol=1e-1, rtol=1e-1)
print(f"Convolution recovery successful: {conv_success}")

Recovery1 successful: True, Recovery2 successful: True
Addition recovery successful: True
Convolution recovery successful: True
